In [7]:
#!/usr/bin/env python3
import pandas as pd
import numpy as np

# =========================
# INPUT FILES (exact paths)
# =========================
BASELINE_SIMPLEMEAN_SUMMARY = "baseline_ols_ridge_rf_simplemean_summary.csv"
BASELINE_GRAVITY_SUMMARY    = "baseline_gravity_summary.csv"

# DeepDemand random 5-fold CV fold-level metrics
DEEPDEMAND_RANDOM_TRAIN_FOLDS = "../logs/cv_fold_best_train_metrics.csv"
DEEPDEMAND_RANDOM_TEST_FOLDS  = "../logs/cv_fold_best_metrics.csv"

# DeepDemand spatial CV (LONG format you pasted)
DEEPDEMAND_SPATIAL_LONG = "../logs/DeepDemand_spatial_cv_summary.csv"   # <- set this to your long-format file name

# =========================
# FORMATTING
# =========================
def fmt_mgeh(mean, std): return f"{mean:.2f} ({std:.2f})"
def fmt_mae(mean,  std): return f"{mean:.0f} ({std:.0f})"
def fmt_r2(mean,   std): return f"{mean:.3f} ({std:.3f})"

def pack_row(train_mgeh, train_mgeh_sd, train_mae, train_mae_sd, train_r2, train_r2_sd,
             test_mgeh,  test_mgeh_sd,  test_mae,  test_mae_sd,  test_r2,  test_r2_sd):
    return {
        "Train_MGEH": fmt_mgeh(train_mgeh, train_mgeh_sd),
        "Train_MAE":  fmt_mae(train_mae,   train_mae_sd),
        "Train_R2":   fmt_r2(train_r2,     train_r2_sd),
        "Test_MGEH":  fmt_mgeh(test_mgeh,  test_mgeh_sd),
        "Test_MAE":   fmt_mae(test_mae,    test_mae_sd),
        "Test_R2":    fmt_r2(test_r2,      test_r2_sd),
    }

def mean_std(series: pd.Series):
    s = pd.to_numeric(series, errors="coerce").dropna()
    return float(s.mean()), float(s.std(ddof=1))

# =========================
# HELPERS: read "wide" baseline summary rows
# =========================
def read_wide_summary(path: str) -> pd.DataFrame:
    df = pd.read_csv(path)
    # normalize model strings just in case
    df["model"] = df["model"].astype(str)
    df["split"] = df["split"].astype(str)
    return df

def get_from_wide(df: pd.DataFrame, split: str, model: str) -> pd.Series:
    r = df[(df["split"] == split) & (df["model"] == model)]
    if r.empty:
        raise KeyError(f"Missing row in {split=} {model=}. Available models: {sorted(df[df['split']==split]['model'].unique().tolist())}")
    return r.iloc[0]

def pack_from_wide_row(r: pd.Series) -> dict:
    return pack_row(
        r["train_MGEH_mean"], r["train_MGEH_std"],
        r["train_MAE_mean"],  r["train_MAE_std"],
        r["train_R2_mean"],   r["train_R2_std"],
        r["test_MGEH_mean"],  r["test_MGEH_std"],
        r["test_MAE_mean"],   r["test_MAE_std"],
        r["test_R2_mean"],    r["test_R2_std"],
    )

# =========================
# DeepDemand: Random CV from fold CSVs
# =========================
dd_train = pd.read_csv(DEEPDEMAND_RANDOM_TRAIN_FOLDS)
dd_test  = pd.read_csv(DEEPDEMAND_RANDOM_TEST_FOLDS)

# drop the "MEAN±SD" row if present
dd_train["fold_num"] = pd.to_numeric(dd_train["fold"], errors="coerce")
dd_test["fold_num"]  = pd.to_numeric(dd_test["fold"],  errors="coerce")
dd_train = dd_train[dd_train["fold_num"].notna()].copy()
dd_test  = dd_test[dd_test["fold_num"].notna()].copy()

train_mae_mean,  train_mae_sd  = mean_std(dd_train["best_train_MAE"])
train_mgeh_mean, train_mgeh_sd = mean_std(dd_train["best_train_MGEH"])
train_r2_mean,   train_r2_sd   = mean_std(dd_train["best_train_R2"])

test_mae_mean,   test_mae_sd   = mean_std(dd_test["best_val_MAE"])
test_mgeh_mean,  test_mgeh_sd  = mean_std(dd_test["best_val_MGEH"])
test_r2_mean,    test_r2_sd    = mean_std(dd_test["best_val_R2"])

deepdemand_random = pack_row(
    train_mgeh_mean, train_mgeh_sd,
    train_mae_mean,  train_mae_sd,
    train_r2_mean,   train_r2_sd,
    test_mgeh_mean,  test_mgeh_sd,
    test_mae_mean,   test_mae_sd,
    test_r2_mean,    test_r2_sd
)

# =========================
# DeepDemand: Spatial CV from LONG CSV (your pasted format)
# =========================
dd_sp = pd.read_csv(DEEPDEMAND_SPATIAL_LONG)

# Expect: model, split(Train/Test), metric(MAE/MGEH/R2), mean, std, ...
dd_sp["model"]  = dd_sp["model"].astype(str)
dd_sp["split"]  = dd_sp["split"].astype(str)
dd_sp["metric"] = dd_sp["metric"].astype(str)

def get_long(model: str, split: str, metric: str):
    r = dd_sp[(dd_sp["model"] == model) & (dd_sp["split"] == split) & (dd_sp["metric"] == metric)]
    if r.empty:
        raise KeyError(f"Missing in DeepDemand spatial long file: model={model}, split={split}, metric={metric}")
    rr = r.iloc[0]
    return float(rr["mean"]), float(rr["std"])

dd_tr_mgeh, dd_tr_mgeh_sd = get_long("DeepDemand", "Train", "MGEH")
dd_tr_mae,  dd_tr_mae_sd  = get_long("DeepDemand", "Train", "MAE")
dd_tr_r2,   dd_tr_r2_sd   = get_long("DeepDemand", "Train", "R2")

dd_te_mgeh, dd_te_mgeh_sd = get_long("DeepDemand", "Test", "MGEH")
dd_te_mae,  dd_te_mae_sd  = get_long("DeepDemand", "Test", "MAE")
dd_te_r2,   dd_te_r2_sd   = get_long("DeepDemand", "Test", "R2")

deepdemand_spatial = pack_row(
    dd_tr_mgeh, dd_tr_mgeh_sd,
    dd_tr_mae,  dd_tr_mae_sd,
    dd_tr_r2,   dd_tr_r2_sd,
    dd_te_mgeh, dd_te_mgeh_sd,
    dd_te_mae,  dd_te_mae_sd,
    dd_te_r2,   dd_te_r2_sd
)

# =========================
# Baselines: wide summaries
# =========================
simple_df = read_wide_summary(BASELINE_SIMPLEMEAN_SUMMARY)
grav_df   = read_wide_summary(BASELINE_GRAVITY_SUMMARY)

# model name mapping (your files)
MODEL_LINEAR  = "OLS"
MODEL_RIDGE   = "Ridge"
MODEL_RF      = "RF"
MODEL_GRAVITY = "GravityLogLinear"

def baseline_row(split_key: str, model_key: str):
    # gravity lives in grav_df; others in simple_df
    if model_key == MODEL_GRAVITY:
        r = get_from_wide(grav_df, split_key, model_key)
    else:
        r = get_from_wide(simple_df, split_key, model_key)
    return pack_from_wide_row(r)

# Random five-fold CV rows
rand_linear  = baseline_row("kfold5",  MODEL_LINEAR)
rand_ridge   = baseline_row("kfold5",  MODEL_RIDGE)
rand_rf      = baseline_row("kfold5",  MODEL_RF)
rand_gravity = baseline_row("kfold5",  MODEL_GRAVITY)

# Spatial CV rows
sp_linear    = baseline_row("spatial9", MODEL_LINEAR)
sp_ridge     = baseline_row("spatial9", MODEL_RIDGE)
sp_rf        = baseline_row("spatial9", MODEL_RF)
sp_gravity   = baseline_row("spatial9", MODEL_GRAVITY)

# =========================
# Assemble final table
# =========================
rows = []
index = []

# group header rows (blank metrics)
EMPTY = {"Train_MGEH":"", "Train_MAE":"", "Train_R2":"", "Test_MGEH":"", "Test_MAE":"", "Test_R2":""}

index.append("Random five-fold CV"); rows.append(EMPTY)
index.append("Linear regression");   rows.append(rand_linear)
index.append("Ridge regression");    rows.append(rand_ridge)
index.append("Random forest");       rows.append(rand_rf)
index.append("Gravity (log-linear)");rows.append(rand_gravity)
index.append("DeepDemand");          rows.append(deepdemand_random)

index.append("Spatial CV");          rows.append(EMPTY)
index.append("Linear regression ");  rows.append(sp_linear)
index.append("Ridge regression ");   rows.append(sp_ridge)
index.append("Random forest ");      rows.append(sp_rf)
index.append("Gravity (log-linear) ");rows.append(sp_gravity)
index.append("DeepDemand ");         rows.append(deepdemand_spatial)

out = pd.DataFrame(rows, index=index)

# column order exactly as requested
out = out[["Train_MGEH","Train_MAE","Train_R2","Test_MGEH","Test_MAE","Test_R2"]]

out_file = "baseline_summary_final.csv"
out.to_csv(out_file)
print(f"Saved: {out_file}")
print(out)

Saved: baseline_summary_final.csv
                         Train_MGEH    Train_MAE       Train_R2  \
Random five-fold CV                                               
Linear regression      85.09 (0.36)   13326 (67)  0.204 (0.006)   
Ridge regression       85.11 (0.39)   13332 (69)  0.203 (0.006)   
Random forest          26.01 (0.17)    3441 (16)  0.934 (0.001)   
Gravity (log-linear)   65.45 (0.22)    9374 (51)  0.549 (0.005)   
DeepDemand             50.26 (0.67)    6838 (70)  0.746 (0.009)   
Spatial CV                                                        
Linear regression      85.14 (0.90)  13325 (223)  0.199 (0.005)   
Ridge regression       85.15 (0.91)  13328 (224)  0.198 (0.005)   
Random forest          25.38 (0.26)    3340 (47)  0.936 (0.003)   
Gravity (log-linear)   65.41 (0.65)   9363 (139)  0.545 (0.012)   
DeepDemand             51.09 (0.85)   7035 (185)  0.736 (0.016)   

                           Test_MGEH      Test_MAE         Test_R2  
Random five-fold CV      